In [1]:
pip install arviz

     |████████████████████████████████| 1.6MB 6.9MB/s 
     |████████████████████████████████| 768kB 28.7MB/s 
     |████████████████████████████████| 4.7MB 41.0MB/s 
     |████████████████████████████████| 317kB 45.6MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [2]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [4]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [5]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


# General Dawid-Skene 

In [6]:
sm_general = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/general_dawid-skene.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b3b91d5d08ea08e32e8380bfec8b2a00 NOW.


In [7]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((k,k))

dades_sm_gen = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [8]:
pi_ini = np.zeros((w,k,k))
for yw in range(w):
  pi_ini[yw] = np.eye(k)+0.05
  pi_ini[yw] /= np.sum(pi_ini[yw],axis=1)
dades_inicials_gen = {'pi':pi_ini}

In [9]:
optim = sm_general.optimizing(data=dades_sm_gen,init=dades_inicials_gen)
optim

OrderedDict([('tau', array([0.11252923, 0.29709025, 0.59038051])),
             ('pi', array([[[0.85028421, 0.07914283, 0.07057296],
                      [0.21907211, 0.58874502, 0.19218287],
                      [0.14766048, 0.09128907, 0.76105045]],
              
                     [[0.65429652, 0.12651422, 0.21918926],
                      [0.30767353, 0.51727564, 0.17505084],
                      [0.13057862, 0.11073393, 0.75868745]],
              
                     [[0.62403594, 0.08478104, 0.29118302],
                      [0.21374825, 0.60218811, 0.18406364],
                      [0.07408813, 0.1084926 , 0.81741927]],
              
                     [[0.69219166, 0.09323484, 0.2145735 ],
                      [0.19748289, 0.63423896, 0.16827815],
                      [0.09577848, 0.13825913, 0.76596239]],
              
                     [[0.55518691, 0.16259694, 0.28221615],
                      [0.22259685, 0.63070878, 0.14669437],
                      [

# Class-Conditional Dawid-Skene 

In [10]:
sm_cond = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/conditional_dawid-skene.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ebe8ab115e5074b7e42906e44eb05239 NOW.


In [11]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((3,3))

dades_sm_cond = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [12]:
pi_ini = 0.9*np.ones((w,k))
dades_inicials_cond = {'pi_k':pi_ini}

In [13]:
optim_cond = sm_cond.optimizing(data=dades_sm_cond,init=dades_inicials_cond)
optim_cond

OrderedDict([('tau', array([0.11641835, 0.29063652, 0.59294513])),
             ('pi_k', array([[0.83807799, 0.59836365, 0.75733106],
                     [0.65031908, 0.52271787, 0.75633185],
                     [0.62282182, 0.60905796, 0.81679091],
                     [0.69725956, 0.6393832 , 0.76562285],
                     [0.54351267, 0.6353455 , 0.79291203],
                     [0.58407672, 0.58132163, 0.81153748],
                     [0.6299252 , 0.60938472, 0.82000539],
                     [0.69704586, 0.55701376, 0.82369476],
                     [0.6969647 , 0.61527891, 0.82693834],
                     [0.60184114, 0.53640816, 0.79043941],
                     [0.81728851, 0.64673394, 0.78434712],
                     [0.61279838, 0.57003476, 0.80593495],
                     [0.71977342, 0.63623496, 0.78859145],
                     [0.69825348, 0.63027463, 0.85768064],
                     [0.8148269 , 0.62591781, 0.8263993 ],
                     [0.593483  , 0.6036

# Homogeneous Dawid - Skene

In [14]:
sm_homo = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/homogeni_dawid-skene.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4543859a14fbad94a494d1c3eeb41537 NOW.


In [15]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((3,3))

dades_sm_hom = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [16]:
pi_ini = 0.9*np.ones(w)
dades_inicials_hom = {'pi_k':pi_ini}

In [17]:
optim_homo = sm_homo.optimizing(data=dades_sm_hom,init=dades_inicials_hom)
optim_homo

OrderedDict([('tau', array([0.11989646, 0.258313  , 0.62179053])),
             ('pi_k',
              array([0.71897477, 0.68478314, 0.73840626, 0.71496385, 0.72227269,
                     0.72119119, 0.73620445, 0.73588733, 0.74948245, 0.69940491,
                     0.74684207, 0.71930607, 0.73118534, 0.7773559 , 0.77161839,
                     0.73192078, 0.72292088, 0.70172006, 0.73670182, 0.7228838 ])),
             ('log_p_t_C', array([[-22.55569233, -20.07030044,  -5.20544963],
                     [-10.18503731, -19.81390419, -17.11943585],
                     [-20.70959986, -18.05128371,  -8.5642629 ],
                     ...,
                     [-20.48418648,  -8.10386334, -18.62138449],
                     [-20.54204088, -21.54948481,  -5.35990696],
                     [-22.36079725, -21.59325379,  -3.56548853]])),
             ('t_C', array([[2.91665573e-08, 3.50168561e-07, 9.99999621e-01],
                     [9.98961569e-01, 6.57332379e-05, 9.72697403e-04],
   